In [1]:
import pandas as pd
from os.path import join

In [2]:
data_path = join("..", "..", "Dataset-1", "selfie_dataset.txt")
image_path = join("..", "..", "Dataset-1", "images")

In [3]:
headers = [
    "image_name", "score", "partial_faces" ,"is_female" ,"baby" ,"child" ,"teenager" ,"youth" ,"middle_age" ,"senior" ,"white" ,"black" ,"asian" ,"oval_face" ,"round_face" ,"heart_face" ,"smiling" ,"mouth_open" ,"frowning" ,"wearing_glasses" ,"wearing_sunglasses" ,"wearing_lipstick" ,"tongue_out" ,"duck_face" ,"black_hair" ,"blond_hair" ,"brown_hair" ,"red_hair" ,"curly_hair" ,"straight_hair" ,"braid_hair" ,"showing_cellphone" ,"using_earphone" ,"using_mirror", "braces" ,"wearing_hat" ,"harsh_lighting", "dim_lighting"
]
df_image_details = pd.read_csv(data_path, names=headers, delimiter=" ")
df_image_details.head(3)

,image_name,score,partial_faces,is_female,baby,child,teenager,youth,middle_age,senior,...,curly_hair,straight_hair,braid_hair,showing_cellphone,using_earphone,using_mirror,braces,wearing_hat,harsh_lighting,dim_lighting
0,00a454da495e11e28a7322000a1fa414_6,3.901,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,00cddb96ac4c11e3a30212279ba1b65f_6,4.385,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,01cdd7aa1a1a11e2aaa822000a1fb0dd_6,4.243,-1,1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [4]:
needed_columns = [
    'image_name',
    'is_female',
    'baby',
    'child',
    'teenager',
    'youth',
    'middle_age',
    'senior'
]

In [6]:
df_image_details = df_image_details[needed_columns]
df_image_details = df_image_details[df_image_details.is_female != 0]
df_image_details.replace(to_replace=-1, value=0, inplace=True)

In [7]:
image_names = df_image_details.image_name.values.copy()
image_attrs = df_image_details[needed_columns[1:]].values.copy()

In [8]:
image_paths = [join(image_path, iname) + '.jpg' for iname in image_names]

In [ ]:
from keras.utils import Sequence
import numpy as np
import cv2

In [ ]:
cv2.__version__

In [ ]:
class ImageGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        # read your data here using the batch lists, batch_x and batch_y
        x = [self.read_image(filename) for filename in batch_x] 
        y = [atrributes for atrributes in batch_y]
        yield np.array(x), np.array(y)
    
    def read_image(self, fname):
        im = cv2.imread(fname)
        return im;

In [ ]:
train_gen = ImageGenerator(image_paths, image_attrs, batch_size=32)
test_gen = ImageGenerator(image_paths, image_attrs, batch_size=32)

# Training Model

In [ ]:
from keras.applications import resnet50
from keras.layers import Dense
from keras.models import Model
from keras.utils import load_model

In [ ]:
from keras.applications import resnet50
from keras.layers import Dense

In [ ]:
model_rnet = 